In [1]:
import pandas as pd
import geopandas as gpd
import mio
from shapely.geometry import Polygon
from matplotlib import pyplot as plt

In [2]:
# make buffered grid of komb
gdf = gpd.read_file('tif/_grid25.tab')
geoms = []
names = []
for ind, row in gdf.iterrows():
    geom = row.geometry.buffer(200)
    name = row.Block
    geoms.append(geom)
    names.append(name)
gdfn = gpd.GeoDataFrame({'komb_name':names}, geometry=geoms)
mio.write_tab(gdfn, 'tif/_grid25_buffer.tab')

259 row(s) written to mapinfo file.


In [3]:
def get_tif_list(komb_name, komb_grid, tif_grid):
    rect = gpd.read_file(komb_grid)
    rect = rect[rect.komb_name == komb_name]
    bounds = [int(v) for v in rect.total_bounds]
    tif_grid = gpd.read_file(tif_grid)
    del tif_grid['index']
    del rect['index']
    res = gpd.sjoin(tif_grid, rect)
    return bounds, list(res.name)
# get_tif_list('Komb1091', 'tif/_grid25_buffer.tab', 'tif/_grid.tab')

In [4]:
def make_komb(komb_name, komb_grid, tif_grid):
    bounds, tifs = get_tif_list(komb_name, komb_grid, tif_grid)
    x0, y0, x1, y1 = bounds
    empty = pd.DataFrame(columns=range(x0, x1), index=range(y1, y0, -1), data=0, dtype=int)
    for i, tif_name in enumerate(tifs):
        tif = mio.read_raster('tif/' + tif_name + '.tif')[0]
        cut = tif.loc[y1:y0,x0:x1]
        empty = empty.add(cut,fill_value=0)
    empty = empty.sort_index(ascending=False)
    mio.write_raster(empty.fillna(0).astype('uint16'), f'komb_alt/{komb_name}_alt.tif')

In [ ]:
%%time
df = gpd.read_file('tif/_grid25_buffer.tab')
iall = len(df)
for ind, row in df.iterrows():
    mio.show_perc(ind, iall, 1)
    make_komb(row.komb_name, 'tif/_grid25_buffer.tab', 'tif/_grid.tab')

0.0% 0.39% 0.77% 1.16% 